In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

import theano
import numpy as np
import matplotlib.pyplot as plt
from gp import GaussianProcess, RBFKernel

In [3]:
debug = True
if debug:
    theano.config.mode = "FAST_COMPILE"
    theano.config.optimizer = "fast_compile"
    theano.config.exception_verbosity = "high"
else:
    theano.config.mode = "FAST_RUN"
    theano.config.optimizer = "fast_run"

In [4]:
def secret_function(x, noise=0.0):
    return x * np.sin(x) + noise * np.random.normal(size=x.shape)

In [5]:
def as_shared(x, name, borrow=True):
    return theano.shared(value=x, name=name, borrow=borrow)

In [6]:
def print_mean_squared_error(gp, x, y):
    mu = gp.compute_mean(x)
    error = np.average((mu - y)**2)
    print("mean squared error", error)

In [7]:
def print_log_likelihood(gp, x, y):
    training_data = {gp.X: x, gp.Y: y}
    print("log likelihood", np.asscalar(gp.log_likelihood.eval(training_data)))

In [8]:
def print_hyperparameters(gp):
    for param in gp.hyperparameters:
        print(param.name, param.get_value())

In [9]:
def plot_variance(gp, x, y, title=None, std_factor=1.0):
    mu = gp.compute_mean(x)
    std = gp.compute_standard_deviation(x) * std_factor

    plt.figure(figsize=(12, 6))
    samples_plt, = plt.plot(
        gp.X_train.get_value(), gp.Y_train.get_value(),
        "bs", ms=4, label="Sampled points")
    y_plt, = plt.plot(x, y, "k--", label="Ground truth")
    mean_plt, = plt.plot(x, mu, "r", label="Estimate")
    std_plt = plt.gca().fill_between(
        x.flat, (mu - 3 * std).flat, (mu + 3 * std).flat,
        color="#dddddd", label="Three standard deviations")
    plt.axis([-4, 4, -4, 4])
    plt.title("Gaussian Process Estimate" if title is None else title)
    plt.legend(handles=[samples_plt, y_plt, mean_plt, std_plt])
    plt.show()

In [10]:
# Training data.
X = np.random.uniform(-4, 4, 10).reshape(-1, 1)
X = np.vstack([X, X, X, X])
X = as_shared(X, "X")
Y = as_shared(secret_function(X.get_value(), noise=1e-1), "Y")

In [11]:
# Test data.
x = as_shared(np.linspace(-4, 4, 1000).reshape(-1, 1), "x")
y = as_shared(secret_function(x.get_value(), noise=0.0), "y")

In [ ]:
# Hyperparameters.
sigma_s = as_shared(np.random.random(), "sigma_s")
sigma_n = as_shared(np.random.random() * 1e-1, "sigma_n")
length_scale = as_shared(np.random.random(), "length_scale")

In [ ]:
# Construct GP.
kernel = RBFKernel(length_scale, sigma_s, sigma_n)
gp = GaussianProcess(kernel, sigma_n)

In [ ]:
gp.fit(X, Y, skip_optimization=True)

In [ ]:
gp.compile()

In [ ]:
plot_variance(gp, x.get_value(), y.get_value(), title="Before Optimization")
print_mean_squared_error(gp, x.get_value(), y.get_value())
print_log_likelihood(gp, X.get_value(), Y.get_value())
print_hyperparameters(gp)

In [ ]:
gp.fit(X, Y)

In [ ]:
plot_variance(gp, x.get_value(), y.get_value(), title="After Optimization")
print_mean_squared_error(gp, x.get_value(), y.get_value())
print_log_likelihood(gp, X.get_value(), Y.get_value())
print_hyperparameters(gp)